In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from PIL import Image
import os
import pandas as pd
from torch.utils.data import DataLoader, Dataset, TensorDataset


def load_images(image_path, transform = None):
    image_dict = {}
    for img in os.listdir(image_path):
        idx = img.split('.')[0]
        abs_path = os.path.join(image_path, img)
        img = Image.open(abs_path).convert('RGB')
        img = transform(img)
        image_dict[idx] = img
    return image_dict

dataset_path = '../../data/nutrition5k_reconstructed/'
prepared_path = './data'

image_path = os.path.join(dataset_path, 'images')
train_labels = os.path.join(prepared_path, 'train_labels.csv')
val_labels= os.path.join(prepared_path, 'val_labels.csv')
test_labels = os.path.join(prepared_path, 'test_labels.csv')
mass_inputs = os.path.join(prepared_path, 'mass_inputs.csv')

transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor()
])

train_images = load_images(image_path, transform)
print(train_images['dish_1561666849'].shape)


def load_masses(file_path):
    mass_dict = {}
    df = pd.read_csv(file_path)
    for idx, row in df.iterrows():
        mass_dict[row['id']] = torch.tensor(row['mass'])
        
    return mass_dict

mass_dict = load_masses(mass_inputs)
print(mass_dict['dish_1561666849'])


torch.Size([3, 299, 299])
tensor(298.)


In [29]:
# Load the DataFrame once
df = pd.read_csv(train_labels)

# Extract IDs
ids = df['id'].values

# Prepare inputs and labels
inputs = torch.stack([train_images[idx] for idx in ids])
labels = torch.tensor(df.drop(columns=['id', 'mass']).values, dtype=torch.float32)

masses = torch.tensor([mass_dict[idx] for idx in ids], dtype=torch.float32)
dataset = TensorDataset(inputs, masses, labels)

# Create the DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

print(inputs.size())
print(masses.size())
print(labels.size())

# print the shape of the first batch
for batch in dataloader:
    print(batch[0].size())
    print(batch[1].size())
    print(batch[2].size())
    break

torch.Size([2755, 3, 299, 299])
torch.Size([2755])
torch.Size([2755, 4])
torch.Size([32, 3, 299, 299])
torch.Size([32])
torch.Size([32, 4])
